In [2]:
%pip install gradio --quiet


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
import json
import pprint
ml_model = pickle.load(open('ml_predictor.pkl', 'rb'))
with open("spec.json") as f:
    spec=json.load(f)

pprint.pprint(spec)

{'Air temperature _K_': {'dtype': 'float', 'max': 304.5, 'min': 295.3},
 'Process temperature _K_': {'dtype': 'float', 'max': 313.8, 'min': 305.7},
 'Rotational speed _rpm_': {'dtype': 'float', 'max': 2886.0, 'min': 1168.0},
 'Tool wear _min_': {'dtype': 'float', 'max': 253, 'min': 0},
 'Torque _Nm_': {'dtype': 'float', 'max': 76.6, 'min': 3.8},
 'Type': {'dtype': 'int', 'max': 2, 'min': 0}}


In [ ]:
import numpy as np
import pandas as pd
import gradio as gr
headers=['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
#['Type', 'Rotational speed _rpm_', 'Tool wear _min_', 'Strain _minNm_','Power _W_', 'Temperature Difference _K_'],
def predict(air_temperature, process_temperature, rotational_speed, torque, tool_wear, machine_type):
    print(air_temperature)
    print(process_temperature)
    print(rotational_speed)
    print(tool_wear)
    print(torque)
    print(machine_type)
    df = pd.DataFrame(columns=['Type', 'Rotational speed _rpm_', 'Tool wear _min_', 'Strain _minNm_','Power _W_', 'Temperature Difference _K_'])
    df['Type'] = [machine_type]
    df['Rotational speed _rpm_']=[rotational_speed]
    df['Tool wear _min_']=[tool_wear]
    df['Strain _minNm_'] = df['Tool wear _min_'] * torque 
    df['Power _W_'] =  df['Rotational speed _rpm_'] * torque * 2 * np.pi / 60
    df['Temperature Difference _K_'] = [process_temperature - air_temperature]
    """
    # 1,300.3,311.2,1381,65.2,136,8867.2,9429.09241468031,10.899999999999977
    # 1,1381,136,8867.2,9429.09241468031,10.899999999999977
    # result 1,0,0,1,0,0
    df['Type'] = [1]
    df['Rotational speed _rpm_']=[1381]
    df['Tool wear _min_']=[136]
    df['Strain _minNm_'] = [8867.2] 
    df['Power _W_'] =  [9429.09241468031]
    df['Temperature Difference _K_'] = [10.899999999999977]    
    """
    y_pred=ml_model.predict(df)
    y_pred=pd.DataFrame(y_pred, columns=['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'])
    # when any failed, mark machine Failure as failed
    y_pred["Machine failure"] = np.where(y_pred["TWF"]==1 | y_pred["HDF"]==1 | y_pred["PWF"]==1 | y_pred["OSF"]==1 | y_pred["RNF"]==1, 1, 0)
    result = {}
    for c in y_pred.columns:
        result[c]=np.where(y_pred[c]==1,True, False)
    maintenance="Need Maintenance" if y_pred["Machine failure"].sum()>0 else "No action needed"
    return result, maintenance

demo = gr.Interface(
    fn=predict,
    inputs=[
            gr.Slider(100, 350, label="Air temperature"),
            gr.Slider(100, 350, label="Process temperature"),
            gr.Number(label="Rotational speed"),
            gr.Number(label="Torque"),
            gr.Number(label="Tool wear"),
            gr.Radio(["Low", "Medium", "High"], label="Type"),
            ],
    outputs=[gr.Dataframe(datatype=["str","bool"], row_count=5, col_count=2, show_fullscreen_button=True), 
             gr.components.Textbox(label="Action")]
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


100
100
0
0
0
None


- TWF fail if Tool Wear (Min) > 198
- OSF fail if Strain (minNm) >= 11019.2
- HDF Fail if Rotational Speed (rpm) <= 1379 and Temperature Difference <=8.599999999999966
- PWF fail if Power _W_ <= 3521.78819652723 or Power _W_ >= 8998.024014705741